In [1]:
import pandas as pd
import numpy as np

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

from dateutil import parser
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


In [2]:
database_username = 'root'
database_password = 'yash'
database_ip       = '127.0.0.1'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [3]:
df = pd.read_sql('SELECT * FROM marketsmojorecos', con=database_connection)
del df['srno']
del df['index']


In [4]:
df

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Prabhudas Lilladher,Colgate Palmol. (I),1248.25,1274.35,2019-09-03,1159.00
1,Axis Direct,Larsen & Toubro,1310.90,1328.1,2019-09-03,1487.00
2,Nirmal Bang,Britannia Inds,2695.00,2704.8,2019-09-03,3220.00
3,Sushil Finance,Sahyadri Industries,147.00,145.5,2019-09-03,220.00
4,Geojit Financial Services,Tata Chemicals,580.95,579.06,2019-08-30,531.00
...,...,...,...,...,...,...
22115,Prabhudas Lilladher,BPCL,493.00,378.64,2019-09-05,466.00
22116,Axis Direct,Bajaj Finance,4056.35,3243.58,2019-09-04,3591.00
22117,Geojit Financial Services,Hindalco,207.60,179.21,2019-09-04,195.00
22118,Geojit Financial Services,HDFC,2368.00,2096.68,2019-09-04,2284.00


In [5]:
df.drop_duplicates(inplace = True)
df

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Prabhudas Lilladher,Colgate Palmol. (I),1248.25,1274.35,2019-09-03,1159.00
1,Axis Direct,Larsen & Toubro,1310.90,1328.1,2019-09-03,1487.00
2,Nirmal Bang,Britannia Inds,2695.00,2704.8,2019-09-03,3220.00
3,Sushil Finance,Sahyadri Industries,147.00,145.5,2019-09-03,220.00
4,Geojit Financial Services,Tata Chemicals,580.95,579.06,2019-08-30,531.00
...,...,...,...,...,...,...
21738,Motilal Oswal,Multi Commodity Exch,1122.00,864.94,2019-09-05,1000.00
21739,Axis Direct,Bajaj Finance,4056.35,3243.58,2019-09-04,3591.00
21740,Geojit Financial Services,Hindalco,207.60,179.21,2019-09-04,195.00
21741,Geojit Financial Services,HDFC,2368.00,2096.68,2019-09-04,2284.00


In [6]:
start_date = '2017-01-01'
end_date = '2017-12-31'
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()

In [7]:
mask = (df['predict_date'] > date_object1) & (df['predict_date'] <= date_object2)

# 2017 data

In [8]:
df1 = df.loc[mask]
df1 = df1.reset_index()
#del df1['level_0']
#del df1['srno']
del df1['index']

In [9]:
print("No of Brokers in 2017 :",df1['broker'].nunique()) 
s1=len(df1)

No of Brokers in 2017 : 62


In [10]:
df3=pd.DataFrame()
for i in range(0,len(df1)): 
    sym=df1.at[i, 'company_name']
    sym = sym.lower()
    sym = sym.replace(" ","")
    sym = sym.replace(".","")
    sym = sym.replace("(i)","")
    sym = sym.replace("(","")
    sym = sym.replace(")","")
    sym = sym.replace("-","")
    sqlrun = "SELECT SYMBOL FROM symbolsmapped WHERE recoName = '"+sym+"'"
    df3 = df3.append(pd.read_sql(sqlrun, con=database_connection),ignore_index=True)

In [11]:
len(df1)


6291

In [12]:
df4 =pd.concat([df1,df3],axis=1)
df4

,broker,company_name,current_price,recomended_buying,predict_date,target_price,SYMBOL
0,Indiabulls Ventures,L&T Finance Holdings,91.80,163.54,2017-08-11,230.00,L&TFH
1,Motilal Oswal,Tech Mahindra,696.90,388.00,2017-06-29,465.00,TECHM
2,Ventura,APL Apollo Tubes,1278.85,1525.00,2017-06-27,2647.6,APLAPOLLO
3,ICICIdirect.com,Power Grid Corpn.,199.65,208.00,2017-06-01,230.00,POWERGRID
4,Edelweiss,SKF India,1881.40,1565.00,2017-05-30,1716.00,SKFINDIA
...,...,...,...,...,...,...,...
6286,HDFC Securities,Bank Of Baroda,92.75,149.00,2017-01-03,195.00,BANKBARODA
6287,ICICIdirect.com,SBI,272.20,245.4,2017-01-03,310.00,SBIN
6288,ICICIdirect.com,GAIL India,129.90,187.13,2017-01-03,213.58,GAIL
6289,Motilal Oswal,PVR,1482.00,1167.00,2017-01-03,1429.00,PVR


In [13]:
uniqueBrokers = df4.broker.unique()
uniqueBrokers

array(['Indiabulls Ventures', 'Motilal Oswal', 'Ventura',
       'ICICIdirect.com', 'Edelweiss', 'KRChoksey', 'Way2Wealth',
       'Investment Imperative', 'Kotak Securities', 'AnandRathi',
       'Prabhudas Lilladher', 'Reliance Securities', 'Angel Broking',
       'HDFC Securities', 'Choice Equity Broking', 'JM Financial',
       'Arihant Capital Markets', 'Sptulsian.com', 'Sharekhan',
       'Axis Direct', 'Cholamandalam Securities', 'Centrum',
       'Nirmal Bang', 'CD Equisearch', 'Bonanza', 'Dolat Capital',
       'Powermywealth.com', 'YES Securities', 'LKP Securities Limited',
       'Ajcon Global', 'IIFL', 'Geojit Financial Services',
       'Joindre Capital Services', 'Dalmia Securities', 'Phillip Capital',
       'SMC Research', 'Khambatta Securities',
       'Rudra Shares & Stocks Brokers', 'Elara Capital',
       'Shahina Mukadam', 'Hem Securities', 'Enochventures.com',
       'Rajatkbose.com', 'Antique Stock Broking', 'BP Wealth',
       'SPA Securities', 'East India Secur

In [14]:
#del brokerPerformace
brokerPerformace = pd.DataFrame(columns=['Name','Hit Ratio','Avg. Growth','Total Recos'])
brokerPerformace

,Name,Hit Ratio,Avg. Growth,Total Recos


In [16]:
for brokerName in uniqueBrokers :
    dfbrokerRecos = df4.loc[df4['broker'] == brokerName]
    print('brokerName : ',brokerName)
    hits=0
    growthSum=0
    total=0
    for index, row in dfbrokerRecos.iterrows():
        #print(row)
        sym = row['SYMBOL']
        print(type(sym))

        avgGrowth=0
        targetp=0
        recbuying=0
        maxval=0
        growthperc=0
        
        if type(sym) == str :
            mydate1 = row['predict_date']#prediction date
            mydate2 = mydate1 + relativedelta(years=1)#date 1 year after prediction
            dbyear1 = "nse"+str(mydate1.year)
            dbyear2 = "nse"+str(mydate2.year)
        
            sqlrun1 = "SELECT * FROM "+dbyear1+" WHERE Date >= '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
            dfprices1 = pd.read_sql(sqlrun1, con=database_connection)
            del dfprices1['index']
        
            sqlrun2 = "SELECT * FROM "+dbyear2+" WHERE Date <= '"+str(mydate2)+"' AND Symbol = '"+sym+"'"
            dfprices2 = pd.read_sql(sqlrun2, con=database_connection)
            del dfprices2['index']
            dfprices = pd.concat([dfprices1,dfprices2],ignore_index=True)

            targetp = float(row['target_price'])
        
            print("SYMBOL : ",sym)
            dfhits = dfprices.loc[dfprices['Closeval'] >= targetp]
            
            total = total + 1
            
            if not dfhits.empty :
                print("HIT!")
                hits = hits + 1
                recbuying = float(row['recomended_buying'])
                maxval = dfprices['Closeval'].max()
                growthperc = (maxval - recbuying) *100 / recbuying
                print('Growth : ',growthperc)
                if np.isnan(growthperc):
                    total = total-1
                else:    
                    if(growthperc < 200):
                        growthSum = growthSum + growthperc
                    else:
                        print("----------------ALERT--------------")
                        high = recbuying * 1.2
                        low = recbuying * 0.8
                        recDate = row['predict_date']
                        print("recDate : ",recDate)
                        dferr = dfprices.loc[dfprices['Date'] == recDate]
                        if not dferr.empty :
                            if dferr['Closeval'].iloc[0] >= low and dferr['Closeval'].iloc[0] <= high :
                                growthSum = growthSum + growthperc
                            else:
                                total = total - 1
                                hits = hits - 1
                        else:
                            total = total - 1
                            hits = hits - 1
            else:
                print("DIDN'T HIT!")
            del dfprices
            del dfhits
    if total != 0:
        hitRatio = int(hits*100 / total)
        avgGrowth = int(growthSum / total)
        print("Broker : ", brokerName, "HitRatio : ", hitRatio, "Avg. Growth : ", avgGrowth, "Total Recos. : ", total)
        brokerPerformace = brokerPerformace.append({'Name':brokerName,'Hit Ratio':hitRatio,'Avg. Growth':avgGrowth,'Total Recos':total}, ignore_index=True)

brokerName :  Indiabulls Ventures
<class 'str'>


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



SYMBOL :  L&TFH
DIDN'T HIT!
<class 'str'>
SYMBOL :  BLUEDART
DIDN'T HIT!
<class 'str'>
SYMBOL :  LICHSGFIN
DIDN'T HIT!
<class 'str'>
SYMBOL :  RECLTD
HIT!
Growth :  999.4103556292611
----------------ALERT--------------
recDate :  2017-10-25
<class 'str'>


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.


SYMBOL :  L&TFH
DIDN'T HIT!
<class 'str'>
SYMBOL :  THERMAX
DIDN'T HIT!
<class 'str'>
SYMBOL :  ORIENTCEM
DIDN'T HIT!
<class 'str'>
SYMBOL :  ADANIPOWER
HIT!
Growth :  47.38853503184714
<class 'str'>
SYMBOL :  ANANTRAJ
DIDN'T HIT!
<class 'str'>
SYMBOL :  UBL
HIT!
Growth :  35.73676546573346
<class 'str'>
SYMBOL :  JUBILANT
HIT!
Growth :  50.1788908765653
<class 'str'>
SYMBOL :  CONCOR
HIT!
Growth :  129.5555142848299
<class 'str'>
SYMBOL :  GMRINFRA
DIDN'T HIT!
<class 'str'>
SYMBOL :  NIITLTD
DIDN'T HIT!
<class 'str'>
SYMBOL :  APOLLOTYRE
HIT!
Growth :  23.198521864093614
<class 'str'>
SYMBOL :  IPCALAB
HIT!
Growth :  59.83565587700183
<class 'str'>
SYMBOL :  THERMAX
DIDN'T HIT!
Broker :  Indiabulls Ventures HitRatio :  41 Avg. Growth :  79 Total Recos. :  17
brokerName :  Motilal Oswal
<class 'str'>
SYMBOL :  TECHM
HIT!
Growth :  84.08505154639175
<class 'str'>
SYMBOL :  SHREECEM
DIDN'T HIT!
<class 'str'>
SYMBOL :  IDEA
HIT!
Growth :  36.23529411764705
<class 'str'>
SYMBOL :  EQUITAS


...

SYMBOL :  REDINGTON
HIT!
Growth :  37.34092431346282
<class 'str'>
SYMBOL :  PAGEIND
HIT!
Growth :  104.78649045521294
<class 'str'>
SYMBOL :  JISLJALEQS
HIT!
Growth :  50.51072522982634
<class 'str'>
SYMBOL :  HDFCBANK
HIT!
Growth :  33.035282745287574
<class 'str'>
SYMBOL :  NCC
HIT!
Growth :  63.99297423887588
<class 'str'>
SYMBOL :  ADANIPORTS
HIT!
Growth :  32.68775872264933
<class 'str'>
SYMBOL :  ESSELPACK
HIT!
Growth :  140.07722007722006
Broker :  IIFL HitRatio :  81 Avg. Growth :  39 Total Recos. :  48
brokerName :  Geojit Financial Services
<class 'str'>
SYMBOL :  ONGC
HIT!
Growth :  22.55158384190641
<class 'str'>
SYMBOL :  CADILAHC
HIT!
Growth :  11.88456878510849
<class 'str'>
SYMBOL :  FEDERALBNK
DIDN'T HIT!
<class 'str'>
SYMBOL :  TCI
HIT!
Growth :  26.699029126213595
<class 'str'>
SYMBOL :  MAHINDCIE
HIT!
Growth :  22.27217000408664
<class 'str'>
SYMBOL :  SUNPHARMA
HIT!
Growth :  44.54283580497173
<class 'str'>
SYMBOL :  TRITURBINE
DIDN'T HIT!
<class 'str'>
SYMBOL :  

SYMBOL :  ONGC
HIT!
Growth :  22.55158384190641
<class 'str'>
SYMBOL :  CADILAHC
HIT!
Growth :  11.88456878510849
<class 'str'>
SYMBOL :  TRITURBINE
DIDN'T HIT!
<class 'str'>
SYMBOL :  ONGC
HIT!
Growth :  22.55158384190641
<class 'str'>
SYMBOL :  CADILAHC
HIT!
Growth :  11.88456878510849
<class 'str'>
SYMBOL :  FEDERALBNK
DIDN'T HIT!
<class 'str'>
SYMBOL :  MAHINDCIE
HIT!
Growth :  22.27217000408664
<class 'str'>
SYMBOL :  SUNPHARMA
HIT!
Growth :  44.54283580497173
<class 'str'>
SYMBOL :  TRITURBINE
DIDN'T HIT!
<class 'str'>
SYMBOL :  FEDERALBNK
DIDN'T HIT!
<class 'str'>
SYMBOL :  AUROPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  VINATIORGA
HIT!
Growth :  45.317990766898646
<class 'str'>
SYMBOL :  DRREDDY
DIDN'T HIT!
<class 'str'>
SYMBOL :  NBCC
HIT!
Growth :  178.63414634146346
<class 'str'>
SYMBOL :  M&MFIN
HIT!
Growth :  179.41001546330438
<class 'str'>
SYMBOL :  GLENMARK
HIT!
Growth :  4.572211918393682
<class 'str'>
SYMBOL :  L&TFH
HIT!
Growth :  19.897959183673468
<class 'str'>
SYMB

SYMBOL :  AUROPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  CIPLA
HIT!
Growth :  20.655979362447006
<class 'str'>
SYMBOL :  TORNTPHARM
HIT!
Growth :  18.221191028615618
<class 'str'>
SYMBOL :  DRREDDY
DIDN'T HIT!
<class 'str'>
SYMBOL :  PRAENG
DIDN'T HIT!
<class 'NoneType'>
<class 'NoneType'>
<class 'str'>
SYMBOL :  AUROPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  DRREDDY
DIDN'T HIT!
<class 'str'>
SYMBOL :  ICICIBANK
HIT!
Growth :  31.53022327101108
<class 'NoneType'>
<class 'str'>
SYMBOL :  SUNPHARMA
DIDN'T HIT!
<class 'NoneType'>
<class 'str'>
SYMBOL :  RAMASTEEL
HIT!
Growth :  115.83333333333331
<class 'str'>
SYMBOL :  IPCALAB
HIT!
Growth :  24.078694817658356
<class 'str'>
SYMBOL :  LUPIN
DIDN'T HIT!
<class 'str'>
SYMBOL :  AUROPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  DRREDDY
DIDN'T HIT!
<class 'str'>
SYMBOL :  AXISBANK
HIT!
Growth :  39.34831460674158
<class 'NoneType'>
<class 'str'>
SYMBOL :  RAMASTEEL
HIT!
Growth :  32.88246268656715
<class 'str'>
SYMBOL :  SUNPHARMA
HIT!
Growth : 

SYMBOL :  UJJIVAN
DIDN'T HIT!
<class 'str'>
SYMBOL :  TVSMOTOR
HIT!
Growth :  56.07355864811132
<class 'str'>
SYMBOL :  ULTRACEMCO
DIDN'T HIT!
<class 'str'>
SYMBOL :  HDFCBANK
HIT!
Growth :  36.27254509018036
<class 'str'>
SYMBOL :  MAHINDCIE
DIDN'T HIT!
<class 'str'>
SYMBOL :  PAPERPROD
HIT!
Growth :  56.08510638297873
<class 'str'>
SYMBOL :  ITDCEM
HIT!
Growth :  45.36423841059602
<class 'str'>
SYMBOL :  SPAL
DIDN'T HIT!
<class 'str'>
SYMBOL :  MOLDTKPAC
HIT!
Growth :  75.91346153846153
<class 'str'>
SYMBOL :  JKIL
HIT!
Growth :  62.623318385650215
<class 'str'>
SYMBOL :  AHLUCONT
HIT!
Growth :  36.329966329966325
<class 'NoneType'>
<class 'str'>
SYMBOL :  BANKBARODA
HIT!
Growth :  4.122340425531915
<class 'str'>
SYMBOL :  ICIL
HIT!
Growth :  22.573964497041423
<class 'str'>
SYMBOL :  IGARASHI
HIT!
Growth :  44.38076416337285
<class 'str'>
SYMBOL :  NIITTECH
HIT!
Growth :  72.87259615384615
<class 'str'>
SYMBOL :  RELIANCE
HIT!
Growth :  109.70427135359078
<class 'str'>
SYMBOL :  BSO

SYMBOL :  LT
HIT!
Growth :  25.348982568233904
<class 'str'>
SYMBOL :  BHARATFIN
HIT!
Growth :  55.76030072971178
<class 'str'>
SYMBOL :  ASIANPAINT
HIT!
Growth :  30.23219653778375
<class 'str'>
SYMBOL :  M&MFIN
HIT!
Growth :  319.51746489016926
----------------ALERT--------------
recDate :  2017-01-24
<class 'str'>
SYMBOL :  ZEEL
HIT!
Growth :  27.01107775132001
<class 'str'>
SYMBOL :  HCLTECH
HIT!
Growth :  18.797387145689967
<class 'str'>
SYMBOL :  TVSMOTOR
HIT!
Growth :  96.06643356643357
<class 'str'>
SYMBOL :  HDFCBANK
HIT!
Growth :  60.940396828527476
<class 'str'>
SYMBOL :  CANBK
HIT!
Growth :  61.34238034109665
<class 'str'>
SYMBOL :  RALLIS
DIDN'T HIT!
<class 'str'>
SYMBOL :  MCDOWELL-N
HIT!
Growth :  90.25217767938784
<class 'str'>
SYMBOL :  YESBANK
HIT!
Growth :  597.6252319109462
----------------ALERT--------------
recDate :  2017-01-19
<class 'str'>
SYMBOL :  AXISBANK
HIT!
Growth :  22.00289375775113
<class 'str'>
SYMBOL :  MARUTI
HIT!
Growth :  71.97122554610054
<class 

In [17]:
brokerPerformace

,Name,Hit Ratio,Avg. Growth,Total Recos
0,Indiabulls Ventures,41,79,17
1,Indiabulls Ventures,41,79,17
2,Motilal Oswal,65,94,775
3,Ventura,42,53,14
4,ICICIdirect.com,78,137,954
...,...,...,...,...
58,India Nivesh,44,10,9
59,Sushil Finance,100,76,2
60,Wallfort Financial,0,0,2
61,GEPL Capital,60,31,5


In [18]:
brokerPerformace.to_sql(con=database_connection, name='brokerperformance2017', if_exists='replace')